In [8]:
import csv
from glob import glob
from pprint import pprint
import numpy as np
import pandas as pd

In [9]:
# Pick a few master-postcode-pairs (which have been augmented with time_sec in Notebook #4).
MPC_PAIRS_FILES = glob("./output/gmaps/master-postcode-pairs-*.csv")

mpc_pairs_all = []

# Import the pairs into dicts
for path in MPC_PAIRS_FILES:
    with open(path) as fp:
        mpc_pairs_all.append(list(csv.DictReader(fp, delimiter="|")))
# Note: The lat and lon are strings instead of floats, but that's ok as we need strings for the API anyways.


# Time Matrix

Using the same idea as the Distance Matrix, we instead find the Google Maps API time taken to travel between each master postcode pairing.

In [10]:
# Reuse the distance matrix from an earlier notebook.
df_mat = pd.read_pickle("./output/distance-matrix.pkl")
df_mat = df_mat.applymap(lambda x: np.NaN).astype(float)  # Change all to NA


In [11]:
# Assign 
for mpc_pairs in mpc_pairs_all:
    for item in mpc_pairs:
        mpc1 = (item["ppv_region"], item["master_postcode_1"])
        mpc2 = (item["ppv_region"], item["master_postcode_2"])
        duration = int(item["time_sec"]) if item["time_sec"] != "inf" else np.inf
        df_mat.loc[mpc1, mpc2] = duration
        df_mat.loc[mpc2, mpc1] = duration # also add across the diagonal


# # Drop all rows and columns which are completely NaN (if testing a sub-sample)
# df_mat = df_mat[df_mat>0].dropna(how="all", axis=0).dropna(how="all", axis=1)

# Set the diagonals to zero.
for i in range(len(df_mat)):
    df_mat.iloc[i, i] = 0 # Set diagonals to zero

display(df_mat)

Johor                                                  \
                   79000   80000   81000   81300   81400   81440   81450   
Johor      79000     0.0  1231.0  1802.0  1190.0  1535.0  3231.0  3086.0   
           80000  1231.0     0.0  2136.0   940.0  1380.0  3667.0  3420.0   
           81000  1802.0  2136.0     0.0  1735.0   995.0  1685.0  1484.0   
           81300  1190.0   940.0  1735.0     0.0  1151.0  3437.0  3191.0   
           81400  1535.0  1380.0   995.0  1151.0     0.0  2343.0  2045.0   
...                  ...     ...     ...     ...     ...     ...     ...   
Terengganu 24060     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           24100     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           24200     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           24300     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
WP Labuan  87000     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

                                          ... Terengganu                  \
                   81500   81550   81600  ...      23200   23300   23400   
Johor      79000  1471.0   458.0  5677.0  ...        NaN     NaN     NaN   
           80000  2247.0  1435.0  5166.0  ...        NaN     NaN     NaN   
           81000  2005.0  1794.0  5126.0  ...        NaN     NaN     NaN   
           81300  2018.0  1300.0  5176.0  ...        NaN     NaN     NaN   
           81400  1828.0  1213.0  4685.0  ...        NaN     NaN     NaN   
...                  ...     ...     ...  ...        ...     ...     ...   
Terengganu 24060     NaN     NaN     NaN  ...     4021.0  3055.0  3288.0   
           24100     NaN     NaN     NaN  ...     3456.0  2444.0  3062.0   
           24200     NaN     NaN     NaN  ...     2784.0  1772.0  2499.0   
           24300     NaN     NaN     NaN  ...     2426.0  1415.0  2142.0   
WP Labuan  87000     NaN     NaN     NaN  ...        NaN     NaN     NaN   

                                                                 WP Labuan  
                   24000   24050   24060   24100   24200   24300     87000  
Johor      79000     NaN     NaN     NaN     NaN     NaN     NaN       NaN  
           80000     NaN     NaN     NaN     NaN     NaN     NaN       NaN  
           81000     NaN     NaN     NaN     NaN     NaN     NaN       NaN  
           81300     NaN     NaN     NaN     NaN     NaN     NaN       NaN  
           81400     NaN     NaN     NaN     NaN     NaN     NaN       NaN  
...                  ...     ...     ...     ...     ...     ...       ...  
Terengganu 24060  1289.0  2248.0     0.0  1348.0  1796.0  2166.0       NaN  
           24100  1571.0  2439.0  1348.0     0.0  1020.0  1389.0       NaN  
           24200  1887.0  2502.0  1796.0  1020.0     0.0   660.0       NaN  
           24300  2178.0  2576.0  2166.0  1389.0   660.0     0.0       NaN  
WP Labuan  87000     NaN     NaN     NaN     NaN     NaN     NaN       0.0  

[432 rows x 432 columns]

In [12]:
# Also, generate the matrix in minutes (instead of seconds), for easy viewing.
df_mat_minfloored = df_mat // 60
display(df_mat_minfloored)

Johor                                                        \
                 79000 80000 81000 81300 81400 81440 81450 81500 81550 81600   
Johor      79000   0.0  20.0  30.0  19.0  25.0  53.0  51.0  24.0   7.0  94.0   
           80000  20.0   0.0  35.0  15.0  23.0  61.0  57.0  37.0  23.0  86.0   
           81000  30.0  35.0   0.0  28.0  16.0  28.0  24.0  33.0  29.0  85.0   
           81300  19.0  15.0  28.0   0.0  19.0  57.0  53.0  33.0  21.0  86.0   
           81400  25.0  23.0  16.0  19.0   0.0  39.0  34.0  30.0  20.0  78.0   
...                ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
Terengganu 24060   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           24100   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           24200   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           24300   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
WP Labuan  87000   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                  ... Terengganu                                            \
                  ...      23200 23300 23400 24000 24050 24060 24100 24200   
Johor      79000  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           80000  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           81000  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           81300  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
           81400  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
...               ...        ...   ...   ...   ...   ...   ...   ...   ...   
Terengganu 24060  ...       67.0  50.0  54.0  21.0  37.0   0.0  22.0  29.0   
           24100  ...       57.0  40.0  51.0  26.0  40.0  22.0   0.0  17.0   
           24200  ...       46.0  29.0  41.0  31.0  41.0  29.0  17.0   0.0   
           24300  ...       40.0  23.0  35.0  36.0  42.0  36.0  23.0  11.0   
WP Labuan  87000  ...        NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   

                       WP Labuan  
                 24300     87000  
Johor      79000   NaN       NaN  
           80000   NaN       NaN  
           81000   NaN       NaN  
           81300   NaN       NaN  
           81400   NaN       NaN  
...                ...       ...  
Terengganu 24060  36.0       NaN  
           24100  23.0       NaN  
           24200  11.0       NaN  
           24300   0.0       NaN  
WP Labuan  87000   NaN       0.0  

[432 rows x 432 columns]

# Export files

In [13]:
# Excel
df_mat.to_excel("./output/time-matrix-sec.xlsx")
df_mat_minfloored.to_excel("./output/time-matrix-minfloored.xlsx")

# Csv - drop one level of the MultiIndex
# Note we must keep the index in the csv (unlike other csv exports so far).
df_mat.index = df_mat.index.droplevel()
df_mat.columns = df_mat.columns.droplevel()
df_mat.to_csv("./output/time-matrix-sec.csv", sep="|")
df_mat_minfloored.index = df_mat_minfloored.index.droplevel()
df_mat_minfloored.columns = df_mat_minfloored.columns.droplevel()
df_mat_minfloored.to_csv("./output/time-matrix-minfloored.csv", sep="|")


In [14]:
display(df_mat)
display(df_mat_minfloored)

,79000,80000,81000,81300,81400,81440,81450,81500,81550,81600,...,23200,23300,23400,24000,24050,24060,24100,24200,24300,87000
79000,0.0,1231.0,1802.0,1190.0,1535.0,3231.0,3086.0,1471.0,458.0,5677.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80000,1231.0,0.0,2136.0,940.0,1380.0,3667.0,3420.0,2247.0,1435.0,5166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81000,1802.0,2136.0,0.0,1735.0,995.0,1685.0,1484.0,2005.0,1794.0,5126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81300,1190.0,940.0,1735.0,0.0,1151.0,3437.0,3191.0,2018.0,1300.0,5176.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81400,1535.0,1380.0,995.0,1151.0,0.0,2343.0,2045.0,1828.0,1213.0,4685.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4021.0,3055.0,3288.0,1289.0,2248.0,0.0,1348.0,1796.0,2166.0,NaN
24100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3456.0,2444.0,3062.0,1571.0,2439.0,1348.0,0.0,1020.0,1389.0,NaN
24200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2784.0,1772.0,2499.0,1887.0,2502.0,1796.0,1020.0,0.0,660.0,NaN
24300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2426.0,1415.0,2142.0,2178.0,2576.0,2166.0,1389.0,660.0,0.0,NaN


,79000,80000,81000,81300,81400,81440,81450,81500,81550,81600,...,23200,23300,23400,24000,24050,24060,24100,24200,24300,87000
79000,0.0,20.0,30.0,19.0,25.0,53.0,51.0,24.0,7.0,94.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80000,20.0,0.0,35.0,15.0,23.0,61.0,57.0,37.0,23.0,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81000,30.0,35.0,0.0,28.0,16.0,28.0,24.0,33.0,29.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81300,19.0,15.0,28.0,0.0,19.0,57.0,53.0,33.0,21.0,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81400,25.0,23.0,16.0,19.0,0.0,39.0,34.0,30.0,20.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67.0,50.0,54.0,21.0,37.0,0.0,22.0,29.0,36.0,NaN
24100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,57.0,40.0,51.0,26.0,40.0,22.0,0.0,17.0,23.0,NaN
24200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,46.0,29.0,41.0,31.0,41.0,29.0,17.0,0.0,11.0,NaN
24300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.0,23.0,35.0,36.0,42.0,36.0,23.0,11.0,0.0,NaN
